In [25]:
! pip install pymongo


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [13]:
from pymongo import MongoClient
import json

In [1]:
def prepare_for_integration(all_data):
    for data in all_data:
        data['iscrizioni'] = data.pop('special_register')
        for entry in data['iscrizioni']:
            entry['number'] = entry.pop('entry_number')
            entry['year'] = entry.pop('inscripted_at')
        data['lauree'] = [{"name": entry['board_name']} for entry in data['iscrizioni']]
    return all_data

In [9]:
from datetime import datetime
import re

def replace_date_fnomceo(data):
    pattern = re.compile(r'\d{2}/\d{2}/\d{4}')
    for element in data:
        date_fields = ["date_of_birth", "last_update_date"]
        for field in date_fields:
            if field in element:
             element[field] = datetime.strptime(element[field], "%d/%m/%Y")
        if "iscrizioni" in element:
            for entry in element["iscrizioni"]:
                if pattern.match(entry["year"]):
                    entry["year"] = datetime.strptime(entry["year"], "%d/%m/%Y")
        if "lauree" in element:
            for entry in element["lauree"]:
                if pattern.match(entry["year"]):
                    entry["year"] = datetime.strptime(entry["year"], "%d/%m/%Y")
        # 
        if "abilitazioni" in element:
            for entry in element["abilitazioni"]:
                if int(entry["year"]) > 1000:
                    entry["year"] = datetime.strptime(entry["year"].strip(), "%Y")
        
        if "specializzazioni" in element:
            for entry in element["specializzazioni"]:
                if pattern.match(entry["year"]):
                    entry["year"] = datetime.strptime(entry["year"], "%d/%m/%Y") if len(date_string) == 10 else None
        #     
    return data

In [10]:
def replace_date_fnofi(data):
    pattern = re.compile(r'\d{2}/\d{2}/\d{4}')
    for element in data:
        date_fields = ["date_of_birth"]
        for field in date_fields:
            if field in element:
             element[field] = datetime.strptime(element[field], "%Y-%m-%dT%H:%M:%S.%fZ")
        if "special_register" in element:
            for entry in element["special_register"]:
                if pattern.match(entry["inscripted_at"]):
                    entry["inscripted_at"] = datetime.strptime(entry["inscripted_at"], "%d/%m/%Y")
    return data

In [14]:
client = MongoClient('localhost', 27017)
db = client.data_managment

In [73]:
fnomceo_data = db.fnomceo_data
with open("./final_data/fnomceo/fnomceo_final_data.json") as f:
    data = json.loads(f.read())
    
standard_data = replace_date_fnomceo(data)

fnomceo_data.insert_many(standard_data)
print(fnomceo_data.count_documents({}))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



************************
{'person_id': '541533', 'surname': 'ATTARDI', 'first_name': 'ENRICO', 'date_of_birth': '12/07/1990', 'birth_place': "L'AQUILA (AQ)", 'province': ' ROMA (RM)', 'full_name': 'Dott. ATTARDI ENRICO', 'prefix': 'Dott.', 'iscrizioni': [{'province': 'ROMA', 'year': '16/03/2017', 'number': '0000062771'}], 'lauree': [{'university_name': 'ROMA TOR VERGATA', 'name': 'MEDICINA E CHIRURGIA ', 'year': '25/10/2016'}], 'abilitazioni': [{'university_name': 'ROMA TOR VERGATA', 'name': 'Medicina e Chirurgia ', 'year': '2016 ', 'round': ' 1'}], 'specializzazioni': [{'university_name': 'FIRENZE', 'specializzazione_name': 'EMATOLOGIA ', 'year': '29/12/2021'}], 'last_update_date': '05/02/2024'}
************************
************************
{'person_id': '395924', 'surname': 'ATTARDI', 'first_name': 'FABIO', 'date_of_birth': '03/03/1958', 'birth_place': 'PALERMO (PA)', 'province': ' AGRIGENTO (AG)', 'full_name': 'DOTT. ATTARDI FABIO', 'prefix': 'DOTT.', 'iscrizioni': [{'province':

In [11]:
alboweb_data = db.alboweb_data
with open("./final_data/alboweb/alboweb_final_data_replaced_profession.json") as f:
    data = json.loads(f.read())

updated_data = replace_date_fnofi(data)

prepared_data = prepare_for_integration(updated_data)

alboweb_data.insert_many(prepared_data)
print(alboweb_data.count_documents({}))

91559


In [12]:
fnofi_data = db.fnofi_data
with open("./final_data/fnofi/fnofi_final_data.json") as f:
    data = json.loads(f.read())
    
updated_data = replace_date_fnofi(data)

prepared_data = prepare_for_integration(updated_data)

fnofi_data.insert_many(prepared_data)
print(fnofi_data.count_documents({}))

48313


In [18]:
import json
integrated_data = db.integrated
# Read the first JSON file
with open("final_data/integrated_data/integrated.json", "r") as data:
    data = json.load(data)
    
integrated_data.insert_many(data)
print(integrated_data.count_documents({}))



496956
